## Проверка GPU

In [1]:
!nvidia-smi
!nvcc --version

from google.colab import drive
drive.mount('/content/drive')

Tue May 31 20:25:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **SberSwap installing**

In [2]:
#@markdown ##*Clone github & download models*
import os 
from google_drive_downloader import GoogleDriveDownloader as gdd
import shutil

if not os.path.exists("sber-swap"):
  !git clone https://github.com/sberbank-ai/sber-swap.git
  %cd sber-swap

# load arcface
  !wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
  !wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/iresnet.py

# load landmarks detector
  !wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/glintr100.onnx
  !wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/scrfd_10g_bnkps.onnx

# load model itself
  !wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/sber-swap-v2.0/G_unet_2blocks.pth

# load super res model
  !wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/super-res/10_net_G.pth
else:
  %cd sber-swap

Cloning into 'sber-swap'...
remote: Enumerating objects: 692, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 692 (delta 33), reused 21 (delta 21), pack-reused 645
Receiving objects: 100% (692/692), 94.00 MiB | 48.00 MiB/s, done.
Resolving deltas: 100% (160/160), done.
/content/sber-swap
--2022-05-31 20:25:43--  https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ai-forever/sber-swap/releases/download/arcface/backbone.pth [following]
--2022-05-31 20:25:43--  https://github.com/ai-forever/sber-swap/releases/download/arcface/backbone.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-

In [ ]:
#@markdown ##*Install required libraries*
if not os.path.exists("sber-swap"):
  !pip install mxnet-cu101mkl
  !pip install onnxruntime-gpu==1.8
  !pip install insightface==0.2.1
  !pip install kornia==0.5.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████▉                     | 240.7 MB 1.3 MB/s eta 0:06:05

In [ ]:
#@markdown ##*Preparation*
import cv2
import torch
import time
import os

from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

In [ ]:
#@markdown ##*Initialize models*

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

In [ ]:
#@markdown ##*Upload source image and video*
 
# choose source image as a photo -- preferable a selfie of a person

target_type = 'image'

source_path = '/content/sber-swap/examples/images/elon_musk.jpg'
target_path = '/content/sber-swap/examples/images/3.jpg'

source_full = cv2.imread(source_path)
crop_size = 224 # don't change this


# check, if we can detect face on the source image

try:    
    source = crop_face(source_full, app, crop_size)[0]
    source = [source[:, :, ::-1]]
    print("Everything is ok!")
except TypeError:
    print("Bad source images")

# read video
if target_type == 'image':
    target_full = cv2.imread(target_path)
    full_frames = [target_full]
else:
    print('It`s not a video')
target = get_target(full_frames, app, crop_size)

In [ ]:
#@markdown ##*Inference*


batch_size =  40

START_TIME = time.time()

final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(full_frames,
                                                                                   source,
                                                                                   target,
                                                                                   netArc,
                                                                                   G,
                                                                                   app,
                                                                                   set_target = False,
                                                                                   crop_size=crop_size,
                                                                                   BS=batch_size)

if use_sr:
    final_frames_list = face_enhancement(final_frames_list, model)

if target_type == 'image':
  result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)
  cv2.imwrite('/content/sber-swap/examples/results/result2.jpg', result)
  print(f'Full pipeline took {time.time() - START_TIME}')
else:
  print('Unluck this is error (video)')

In [ ]:
#@markdown ##*Visualize Image to Image swap*
cv2.imwrite('/content/sber-swap/examples/results/result2.jpg', result)
import matplotlib.pyplot as plt

show_images([source[0][:, :, ::-1], target_full, result], ['Исходное изображение', 'Целевое изображение', 'Результат'], figsize=(20, 15))


# SimSwap installing

In [ ]:
#@title ##*Clone Github & Download models*
%cd /content/
!rm -rf SimSwap &> /dev/null
!git clone https://github.com/neuralchen/SimSwap
!cd SimSwap && git pull

%cd /content/SimSwap
!pip install -q insightface==0.2.1 onnxruntime moviepy
!pip install -q onnxruntime-gpu==1.9.0
!pip install -q imageio==2.4.1
!pip install -q ffmpeg
!pip install -q tf
!pip install -q timm

# import os
# os.chdir("SimSwap")
# !ls

In [ ]:
%cd /content/SimSwap
!mkdir arcface_model &> /dev/null
import os

while not os.path.isfile("./arcface_model/arcface_checkpoint.tar") or os.stat("./arcface_model/arcface_checkpoint.tar").st_size < 766871429:
      !wget -q --show-progress "https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar" -O "./arcface_model/arcface_checkpoint.tar"

while not os.path.isfile("./checkpoints.zip") or os.stat("./checkpoints.zip").st_size < 256461775:
      !wget -q --show-progress "https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip" -O "./checkpoints.zip"

while not os.path.isfile("./antelope.zip") or os.stat("./antelope.zip").st_size < 248024513:
      !wget -q --show-progress -c --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip

while not os.path.isfile("./512.zip") or os.stat("./512.zip").st_size < 282817182:
      !wget -q --show-progress -c --no-check-certificate -P /content/SimSwap/ https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip

while not os.path.isfile("./parsing_model/checkpoint/79999_iter.pth") or os.stat("./parsing_model/checkpoint/79999_iter.pth").st_size < 53289463:
      !wget -q --show-progress -c --no-check-certificate -P /content/SimSwap/parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

!unzip -o -q ./antelope.zip -d ./insightface_func/models/
!unzip -o -q ./checkpoints.zip  -d ./checkpoints
!unzip -o -q ./512.zip  -d ./checkpoints

!rm -rf ./antelope.zip &> /dev/null
!rm -rf ./checkpoints.zip &> /dev/null
!rm -rf ./512.zip &> /dev/null

In [ ]:
#@title ##*Inference*
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap
from util.add_watermark import watermark_image

In [ ]:
!mkdir /content/simswap_img_result &> /dev/null
%cd /content/SimSwap

crop_size = 224

pic_a_path = "/content/sber-swap/examples/images/elon_musk.jpg"
pic_b_path = "/content/sber-swap/examples/images/3.jpg"
output_path = "/content/simswap_img_result/"

START_TIME = time.time()

!python -W ignore test_wholeimage_swapsingle.py --which_epoch latest --isTrain false --crop_size {crop_size} --use_mask  --name people --Arc_path arcface_model/arcface_checkpoint.tar --pic_a_path {pic_a_path} --pic_b_path {pic_b_path} --output_path {output_path} --no_simswaplogo

print(f'Full pipeline took {time.time() - START_TIME}')

In [ ]:
#@title ##*SSIM*

from skimage.metrics import structural_similarity
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

a1 = cv2.imread('/content/examples/res2.jpg')
a2 = cv2.imread('/content/examples/sber2.jpg')
a3 = cv2.imread('/content/examples/sim2.jpg')


def ssim(before, after, shw):
  # Convert images to grayscale
  before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
  after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

  # Compute SSIM between two images
  (score, diff) = structural_similarity(before_gray, after_gray, full=True)
  print("Image similarity", score)

  diff = (diff * 255).astype("uint8")

  thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contours = contours[0] if len(contours) == 2 else contours[1]

  mask = np.zeros(before.shape, dtype='uint8')
  filled_after = after.copy()

  for c in contours:
      area = cv2.contourArea(c)
      if area > 40:
          x,y,w,h = cv2.boundingRect(c)
          cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
          cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
          cv2.drawContours(mask, [c], 0, (0,255,0), -1)
          cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)
  if shw == True:
    cv2_imshow(before)
    cv2_imshow(after)
    cv2_imshow(diff)
    cv2_imshow(mask)
    cv2_imshow(filled_after)
    cv2.waitKey(0)
  return score

ssim(a1, a2, True)
# https://github.com/rahmatnazali/image-copy-move-detection
# https://ece.uwaterloo.ca/~z70wang/publications/ssim.pdf
# https://stackoverflow.com/questions/56183201/detect-and-visualize-differences-between-two-images-with-opencv-python
# https://stackoverflow.com/questions/71567315/how-to-get-the-ssim-comparison-score-between-two-images

# https://github.com/AICoE/pyIFD
# https://github.com/NVlabs/stylegan

In [ ]:
!pip install git+https://github.com/eldritchjs/pyIFD

In [ ]:
import pyIFD
!python pyIFD.pytest.py

/bin/bash: pyIFD.pytest.py: command not found


In [ ]:
#@title ## *ELA algo*
from PIL import Image, ImageChops, ImageEnhance
import sys, os.path

filename = '/content/sample_data/result_whole_swapsingle (2).jpg'
resaved = filename + '.resaved.jpg'
ela = filename + '.ela.png'

im = Image.open(filename)

im.save(resaved, 'JPEG', quality=95)
resaved_im = Image.open(resaved)

ela_im = ImageChops.difference(im, resaved_im)
extrema = ela_im.getextrema()
max_diff = max([ex[1] for ex in extrema])
scale = 255.0/max_diff

ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)

print ("Maximum difference was %d" % (max_diff))
ela_im.save(ela)
ela_im.show()

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# read image
img1 = cv2.imread("/content/sample_data/result_whole_swapsingle (2).jpg")

# set compression and scale
jpg_quality1 = 95
jpg_quality2 = 90
scale = 50

# write img1 at 95% jpg compression
cv2.imwrite("lenna_c95.jpg", img1, [cv2.IMWRITE_JPEG_QUALITY, jpg_quality1])

# read compressed image
img2 = cv2.imread("lenna_c95.jpg")

# get absolute difference between img1 and img2 and multiply by scale
diff1 = scale * cv2.absdiff(img1, img2)

# write img2 at 90% jpg compression
cv2.imwrite("lenna_c90.jpg", img2, [cv2.IMWRITE_JPEG_QUALITY, jpg_quality2])

# read compressed image
img3 = cv2.imread("lenna_c90.jpg")

# get absolute difference between img1 and img2 and multiply by scale
diff2 = scale * cv2.absdiff(img2, img3)

# write result to disk
cv2.imwrite("lenna_ela_95.jpg", diff1)
cv2.imwrite("lenna_ela_90.jpg", diff2)

# display it
cv2_imshow(img1)
cv2_imshow(diff1)
cv2_imshow(diff2)
cv2.waitKey(0)